# TileDB OME-Zarr Conversion

In [1]:
import os
import cv2
import matplotlib.pylab as pylab
import tiledb


from tiledb.bioimg.converters.ome_zarr import OMEZarrConverter
from tiledb.bioimg.openslide import TileDBOpenSlide

## Convert OME-Zarr to TileDB Slide

In [2]:
src = "../tests/data/CMU-1-Small-Region.ome.zarr"
dest = src + ".tiledb"
if not os.path.exists(dest):
    OMEZarrConverter.to_tiledb(src, dest, level_min=0)

StopIteration: 

## Slide Group Contents

In [ ]:
a = tiledb.open(os.path.join(dest,"l_0.tdb"))
a.schema

## TileDB Slide Information

In [ ]:
slide = TileDBOpenSlide(dest)
print("level_count:", slide.level_count)
print("dimensions:", slide.dimensions)
print("level_dimensions:", slide.level_dimensions)
print("level_downsamples:", slide.level_downsamples)
print("levels:", slide.levels)
print("group_properties:", slide.properties)

## Reading Slide whole levels

In [ ]:
img3d = slide.read_level(0)
norm_img3d = cv2.normalize(src=img3d, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
pylab.imshow(norm_img3d)

## Read a region of a level

In [ ]:
img3d = slide.read_region((200,200), 0, slide.dimensions)
norm_img3d = cv2.normalize(src=img3d, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

In [ ]:
pylab.imshow(norm_img3d)

## Read level properties

In [ ]:
print(slide.level_properties(0))